<h1> Bangkok Housing Analysis</h1>

This project will analyse the districts and neighbourhood of Bangkok in term of the available facility in order to help people to decide the the suitable area to live in. This will be based on transport, restaurant, supermarket and other point of interest.

<h2>Import Relevant Libraries</h2>

<h3> Install Libraries </h3>

In [2]:
import sys
!conda install --yes --prefix {sys.prefix} numpy pandas  matplotlib 

!conda install -c conda-forge geopy beautifulsoup4 scikit-learn folium=0.5.0 --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chitt\Anaconda3

  added / updated specs:
    - matplotlib
    - numpy
    - pandas


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> pkgs/main::conda-4.8.3-py37_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chitt\Anaconda3

  added / updated specs:
    - beautifulsoup4
    - folium=0.5.0
    - geopy
    - scikit-learn


The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


<h3> Import Libraries </h3>

In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


<h2> Gather Bangkok district data </h2>

<h3>Import data from wikipedia using Beautiful Soup</h3>

In [22]:
URL = "https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

print(soup.prettify()) 

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of districts of Bangkok - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"16c3ee14-7edf-42d2-b997-f600a166b266","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_districts_of_Bangkok","wgTitle":"List of districts of Bangkok","wgCurRevisionId":961269901,"wgRevisionId":961269901,"wgArticleId":29995815,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Articles containing Thai-language text","Districts of Bangkok","Thail

<h3>Collect the data from import page</h3>

In [361]:
postalCodes = [];
districts= [];
subdistricts = [];
latitude = [];
longitude = [];
population=[];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if columnNum == 1:
            districts.append(cell.string);   
            columnNum = 2
        elif columnNum == 2 :    
            columnNum = 3
        elif columnNum == 3 :
            postalCodes.append(cell.string);      
            columnNum = 4
        elif columnNum == 4 : 
            columnNum = 5
        elif columnNum == 5 : 
            population.append(cell.string); 
            columnNum = 6
        elif columnNum == 6 : 
            subdistricts.append(cell.string); 
            columnNum = 7
        elif columnNum == 7 : 
            latitude.append(cell.string); 
            columnNum = 8
        elif columnNum == 8 : 
            longitude.append(cell.string); 
            columnNum = 1
                
print('Data Collected.')

Data Collected.


<h3>Test Data Collection </h3>

In [362]:
print(postalCodes)

['10150', '10240', '10160', '10220', '10120', '10150', '10260', '10700', '10500', '10800', '10700', '10600', '10240', '10900\n', '10150\n', '10400\n', '10210\n', '10300\n', '10310\n', '10230\n', '10510\n', '10600\n', '10110\n', '10210\n', '10520\n', '10230\n', '10510\n', '10530\n', '10160\n', '10330\n', '10160\n', '10400\n', '10260\n', '10200\n', '10100\n', '10250\n', '10140\n', '10400\n', '10220\n', '10100\n', '10240\n', '10120\n', '10250\n', '10170\n', '10170\n', '10600\n', '10140\n', '10310\n', '10110\n', '10120\n', None]


<h3>Create Initial dataframe</h3>

In [363]:
column_names = ['PostalCode', 'Neighborhood', 'Sub-district','latitude','longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Neighborhood,Sub-district,latitude,longitude


<h3>Put collected data to the Dataframe</h3>

In [364]:
for data in range(len(neighborhoods)):
    if "\n" in postalCodes[data] :
        code = postalCodes[data][:-1]
    else :
        code = postalCodes[data]
    district = districts[data]
    sub_districts_num = subdistricts[data]
    lat = latitude[data]
    lon = longitude[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Neighborhood': district,
                                   'Sub-district': sub_districts_num[:-1],
                                    'latitude': lat[:-1],
                                    'longitude': lon[:-1]}, ignore_index=True)

In [365]:
neighbors

,PostalCode,Neighborhood,Sub-district,latitude,longitude
0,10150,Bang Bon,4,13.6592,100.3991
1,10240,Bang Kapi,2,13.765833,100.647778
2,10160,Bang Khae,4,13.696111,100.409444
3,10220,Bang Khen,2,13.873889,100.596389
4,10120,Bang Kho Laem,3,13.693333,100.5025
5,10150,Bang Khun Thian,2,13.660833,100.435833
6,10260,Bang Na,2,13.680081,100.5918
7,10700,Bang Phlat,4,13.793889,100.505
8,10500,Bang Rak,5,13.730833,100.524167
9,10800,Bang Sue,2,13.809722,100.537222


Bangkok have 50 district in total 

In [366]:
neighbors.shape

(50, 5)

<h2>District Analysis</h2>

In [367]:
district_name = neighbors.loc[49, 'Neighborhood'] # neighborhood name
lat = neighbors.loc[49, 'latitude'] # neighborhood latitude value
lng = neighbors.loc[49, 'longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, 
                                                               lat, 
                                                               lng))


Latitude and longitude values of Yan Nawa are 13.696944, 100.543056.


In [396]:
CLIENT_ID='X42LYOEZGJAW2WD35SA4TB2LESK4BOMBKXPBM4QLGQY40NDZ'
CLIENT_SECRET='FTRC4MGDBI4LDG5VBA0NMAU1DXS14SPIC1UTP3ZILTAJEMTP'
radius = 1500 # define radius
LIMIT = 200 # limit of number of venues returned by Foursquare API
VERSION = '20180323'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ede5df11187ee001bc04c3c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Yan Nawa',
  'headerFullLocation': 'Yan Nawa, Bangkok',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 75,
  'suggestedBounds': {'ne': {'lat': 13.710444013500014,
    'lng': 100.5569252238732},
   'sw': {'lat': 13.683443986499986, 'lng': 100.52918677612682}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b62d9a3f964a52074552ae3',
       'name': 'Tawandang German Brewery (โรงเบียร์เยอรมันตะวันแดง)',
       'contact': {},
       'location': {'address': '462/61 Rama III Rd.',
        'crossStreet': 'Naradhiwas Rajanagarindra Rd.',
        'lat': 13.6967

In [369]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [397]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Tawandang German Brewery (โรงเบียร์เยอรมันตะวั...,Brewery,13.696742,100.545958
1,Au Bon Pain (โอ บอง แปง),Café,13.697340,100.541133
2,Recreational Bangkok Biking,Bike Rental / Bike Share,13.698552,100.544010
3,Hachiban Ramen (ฮะจิบัง ราเมน),Ramen Restaurant,13.697124,100.540969
4,Est.33 by SINGHA,Brewery,13.700792,100.540887
...,...,...,...,...
70,S-Klubchom Club,Soccer Field,13.688103,100.545447
71,สวนอาหารช้างทอง (Chang Thong Garden & Restaurant),Thai Restaurant,13.684143,100.543421
72,ร้านข้าว,Thai Restaurant,13.687050,100.545831
73,แยกสะพานภูมิพล,Intersection,13.687109,100.547727


In [398]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

75 venues were returned by Foursquare.


In [399]:
def getNearbyVenues(names, latitudes, longitudes,search_query):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            search_query,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [400]:
Bangkok_venues = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                search_query = 'restaurant')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [401]:
print(Bangkok_venues.shape)
Bangkok_venues

(3817, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,ขาหมูบางหว้า,13.657136,100.395230,Thai Restaurant
1,Bang Bon,13.6592,100.3991,Irashaimase Japanese Restaurant,13.658358,100.401403,Japanese Restaurant
2,Bang Bon,13.6592,100.3991,เจ๊ไน้ ก๋วยเตี๋ยวเป็ดตุ๋น,13.654137,100.405323,Noodle House
3,Bang Bon,13.6592,100.3991,หมูสะเต๊ะแม่กำไร,13.653925,100.391426,Satay Restaurant
4,Bang Bon,13.6592,100.3991,ส.สุขโภชนา,13.660358,100.404866,Chinese Restaurant
...,...,...,...,...,...,...,...
3812,Yan Nawa,13.696944,100.543056,หมูก๋วยเตี๋ยวแปดริ้ว,13.686867,100.536573,Noodle House
3813,Yan Nawa,13.696944,100.543056,Ciao Italian Restaurant,13.684772,100.544271,Italian Restaurant
3814,Yan Nawa,13.696944,100.543056,Little Pig หมูน้อย,13.684984,100.537980,Thai Restaurant
3815,Yan Nawa,13.696944,100.543056,ตุ๋ย ข้าวต้มบาทเดียว,13.684970,100.538017,Food


In [402]:
print('There are {} uniques categories.'.format(len(Bangkok_venues['Venue Category'].unique())))

There are 84 uniques categories.


In [403]:
# one hot encoding
bangkok_onehot = pd.get_dummies(Bangkok_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangkok_onehot['Neighborhood'] = Bangkok_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangkok_onehot.columns[-1]] + list(bangkok_onehot.columns[:-1])
bangkok_onehot = bangkok_onehot[fixed_columns]

bangkok_onehot

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Buffet,...,Takoyaki Place,Tapas Restaurant,Thai Restaurant,Tonkatsu Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,Yan Nawa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3813,Yan Nawa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3814,Yan Nawa,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3815,Yan Nawa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [404]:
bangkok_onehot.shape

(3817, 85)

In [420]:
bangkok_grouped = bangkok_onehot.groupby('Neighborhood').mean().reset_index()
bangkok_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Buffet,...,Takoyaki Place,Tapas Restaurant,Thai Restaurant,Tonkatsu Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,Bang Bon,0.000000,0.000000,0.000000,0.068182,0.000000,0.000000,0.045455,0.000000,0.000000,...,0.00,0.00,0.113636,0.000000,0.000000,0.022727,0.000000,0.000000,0.00,0.00
1,Bang Kapi,0.000000,0.000000,0.000000,0.020000,0.040000,0.030000,0.000000,0.000000,0.000000,...,0.00,0.00,0.110000,0.000000,0.000000,0.000000,0.010000,0.010000,0.00,0.00
2,Bang Khae,0.000000,0.000000,0.000000,0.116279,0.023256,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.093023,0.000000,0.000000,0.000000,0.000000,0.023256,0.00,0.00
3,Bang Khen,0.000000,0.015385,0.000000,0.076923,0.030769,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.153846,0.000000,0.000000,0.000000,0.030769,0.000000,0.00,0.00
4,Bang Kho Laem,0.000000,0.000000,0.000000,0.060000,0.030000,0.030000,0.020000,0.000000,0.010000,...,0.00,0.00,0.200000,0.000000,0.000000,0.010000,0.020000,0.000000,0.01,0.00
5,Bang Khun Thian,0.000000,0.000000,0.000000,0.050000,0.030000,0.030000,0.000000,0.000000,0.000000,...,0.00,0.00,0.140000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.00
6,Bang Na,0.000000,0.000000,0.000000,0.051282,0.000000,0.076923,0.000000,0.000000,0.000000,...,0.00,0.00,0.230769,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
7,Bang Phlat,0.000000,0.000000,0.000000,0.027778,0.027778,0.041667,0.000000,0.000000,0.000000,...,0.00,0.00,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
8,Bang Rak,0.000000,0.000000,0.000000,0.020000,0.040000,0.030000,0.010000,0.020000,0.010000,...,0.00,0.00,0.140000,0.010000,0.000000,0.010000,0.000000,0.000000,0.00,0.00
9,Bang Sue,0.000000,0.000000,0.000000,0.072727,0.036364,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.236364,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00


In [421]:
bangkok_grouped.shape

(50, 85)

In [422]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in bangkok_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bangkok_grouped[bangkok_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bang Bon----
                  venue  freq
0          Noodle House  0.23
1       Thai Restaurant  0.11
2            Restaurant  0.09
3  Fast Food Restaurant  0.09
4      Asian Restaurant  0.07


----Bang Kapi----
                  venue  freq
0          Noodle House  0.11
1       Thai Restaurant  0.11
2   Japanese Restaurant  0.10
3  Fast Food Restaurant  0.10
4    Som Tum Restaurant  0.08


----Bang Khae----
                 venue  freq
0         Noodle House  0.23
1  Japanese Restaurant  0.14
2     Asian Restaurant  0.12
3      Thai Restaurant  0.09
4                 Café  0.09


----Bang Khen----
                  venue  freq
0       Thai Restaurant  0.15
1          Noodle House  0.11
2            Restaurant  0.08
3      Asian Restaurant  0.08
4  Fast Food Restaurant  0.08


----Bang Kho Laem----
                venue  freq
0     Thai Restaurant  0.20
1        Noodle House  0.11
2  Chinese Restaurant  0.09
3    Asian Restaurant  0.06
4  Seafood Restaurant  0.05


----Bang Khun T

In [408]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [409]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangkok_grouped['Neighborhood']

for ind in np.arange(bangkok_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangkok_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bang Bon,Noodle House,Thai Restaurant,Fast Food Restaurant,Restaurant,Café
1,Bang Kapi,Noodle House,Thai Restaurant,Fast Food Restaurant,Japanese Restaurant,Som Tum Restaurant
2,Bang Khae,Noodle House,Japanese Restaurant,Asian Restaurant,Thai Restaurant,Café
3,Bang Khen,Thai Restaurant,Noodle House,Restaurant,Fast Food Restaurant,Asian Restaurant
4,Bang Kho Laem,Thai Restaurant,Noodle House,Chinese Restaurant,Asian Restaurant,Seafood Restaurant
5,Bang Khun Thian,Thai Restaurant,Noodle House,Restaurant,Steakhouse,Japanese Restaurant
6,Bang Na,Thai Restaurant,Noodle House,Café,Fast Food Restaurant,Restaurant
7,Bang Phlat,Thai Restaurant,Noodle House,Café,Fast Food Restaurant,Restaurant
8,Bang Rak,Café,Thai Restaurant,Noodle House,Seafood Restaurant,Chinese Restaurant
9,Bang Sue,Noodle House,Thai Restaurant,Asian Restaurant,Fast Food Restaurant,Restaurant


In [410]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

bangkok_grouped_clustering = bangkok_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangkok_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 1, 4, 1, 1, 1, 2, 2, 4, 0])

In [411]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangkok_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bangkok_merged = bangkok_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bangkok_merged # check the last columns!

,PostalCode,Neighborhood,Sub-district,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,10150,Bang Bon,4,13.6592,100.3991,4,Noodle House,Thai Restaurant,Fast Food Restaurant,Restaurant,Café
1,10240,Bang Kapi,2,13.765833,100.647778,1,Noodle House,Thai Restaurant,Fast Food Restaurant,Japanese Restaurant,Som Tum Restaurant
2,10160,Bang Khae,4,13.696111,100.409444,4,Noodle House,Japanese Restaurant,Asian Restaurant,Thai Restaurant,Café
3,10220,Bang Khen,2,13.873889,100.596389,1,Thai Restaurant,Noodle House,Restaurant,Fast Food Restaurant,Asian Restaurant
4,10120,Bang Kho Laem,3,13.693333,100.5025,1,Thai Restaurant,Noodle House,Chinese Restaurant,Asian Restaurant,Seafood Restaurant
5,10150,Bang Khun Thian,2,13.660833,100.435833,1,Thai Restaurant,Noodle House,Restaurant,Steakhouse,Japanese Restaurant
6,10260,Bang Na,2,13.680081,100.5918,2,Thai Restaurant,Noodle House,Café,Fast Food Restaurant,Restaurant
7,10700,Bang Phlat,4,13.793889,100.505,2,Thai Restaurant,Noodle House,Café,Fast Food Restaurant,Restaurant
8,10500,Bang Rak,5,13.730833,100.524167,4,Café,Thai Restaurant,Noodle House,Seafood Restaurant,Chinese Restaurant
9,10800,Bang Sue,2,13.809722,100.537222,0,Noodle House,Thai Restaurant,Asian Restaurant,Fast Food Restaurant,Restaurant


In [412]:
# create map
map_clusters = folium.Map(location=[13.77, 100.6], zoom_start=11.45)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(bangkok_merged['latitude'], bangkok_merged['longitude'], bangkok_merged['Neighborhood'], bangkok_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        location=[float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [413]:
Bangkok_school = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'school')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [414]:
print(Bangkok_school.shape)
Bangkok_school

(1172, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Kapi,13.765833,100.647778,Net Design,13.767537,100.644157,School
1,Bang Kapi,13.765833,100.647778,NetDesign Bangkapi,13.767814,100.643950,School
2,Bang Kapi,13.765833,100.647778,Sakura Japanese School,13.766799,100.643451,School
3,Bang Kapi,13.765833,100.647778,CYES โรงเรียนชมอักษร,13.765311,100.642483,School
4,Bang Kapi,13.765833,100.647778,ECC (Thailand),13.766581,100.642461,School
...,...,...,...,...,...,...,...
1167,Yan Nawa,13.696944,100.543056,Barracuda School,13.694963,100.541523,Swim School
1168,Yan Nawa,13.696944,100.543056,สยามกลการ พระรามสาม,13.698452,100.537920,Music School
1169,Yan Nawa,13.696944,100.543056,Yamaha Music School,13.698398,100.537840,Music School
1170,Yan Nawa,13.696944,100.543056,Samit Music School,13.703093,100.535696,Music School


In [415]:
print('There are {} uniques categories.'.format(len(Bangkok_school['Venue Category'].unique())))

There are 41 uniques categories.
